In [1]:
import numpy as np
import math

In [2]:
#electrode matrices
electrode_voltages = np.zeros((8,8))
electrode_positions = np.zeros((8,8,2))


#RGC matrices
on_parasol_responses = np.zeros((2*28,16)) #28 rows in hexagonal grid means 28 rows on a straight, but 56 rows staggered
on_parasol_positions = np.zeros((2*28,16,2)) #32 cols in hex grid means 32 staggered cols = 16 straight cols

off_parasol_responses = np.zeros((2*31,18)) #31 rows in hexagonal grid means 31 rows on a straight, but 62 rows staggered
off_parasol_positions = np.zeros((2*28,18,2)) #35 cols in hex grid means 35 staggered cols = 17 or 18 straight cols

on_midget_responses = np.zeros((2*55,32)) #55 rows in hexagonal grid means 55 rows on a straight, but 110 rows staggered
on_midget_positions = np.zeros((2*55,32,2)) #63 cols in hex grid means 63 staggered cols = 31 or 32 straight cols

off_midget_responses = np.zeros((2*61,35)) #61 rows in hexagonal grid means 61 rows on a straight, but 122 rows staggered
off_midget_positions = np.zeros((2*61,35,2)) #70 cols in hex grid means 70 staggered cols = 35 straight cols


#Bipolar matrices 512x512 area with 2 micron pitch, so 1 neuron every 2 microns, so 256x256 array
on_parasol_bipolar_responses = np.zeros((2*256,128)) #256 rows in hex grid means 256 rows on a straight, but 128 rows staggered
on_parasol_bipolar_positions = np.zeros((2*256,128,2)) #256 cols in a hex grid means 256 staggered cols = 128 straight cols

off_parasol_bipolar_responses = np.zeros((2*256,128)) #256 rows in hex grid means 256 rows on a straight, but 128 rows staggered
off_parasol_bipolar_positions = np.zeros((2*256,128,2)) #256 cols in a hex grid means 256 staggered cols = 128 straight cols

on_midget_bipolar_responses = np.zeros((2*256,128)) #256 rows in hex grid means 256 rows on a straight, but 128 rows staggered
on_midget_bipolar_positions = np.zeros((2*256,128,2)) #256 cols in a hex grid means 256 staggered cols = 128 straight cols

off_midget_bipolar_responses = np.zeros((2*256,128)) #256 rows in hex grid means 256 rows on a straight, but 128 rows staggered
off_midget_bipolar_positions = np.zeros((2*256,128,2)) #256 cols in a hex grid means 256 staggered cols = 128 straight cols

In [3]:
#set positions

#electrodes:
#512 micron patch of retina and 70 micron electrode pitch
#odd rows start at x=0 and go to 0+70*7 = 490
#even rows start at x=512 and go down to 512-70*7 = 22 (x=22 to x=512 by 70)
#y values are evenly spread from y=16 to about y=506 (by 70)

y_positions = np.arange(16,507,70)

i = 0
while(i<np.shape(electrode_positions)[0]): #go through all rows, use np.arrange to evenly space electrodes
    #set y position for this row
    electrode_positions[i,:,1] = np.ones(8)*y_positions[i]
    
    #set x positions for each column in this row
    if(i%2 == 0):
        electrode_positions[i,:,0] = np.arange(22, 513, 70)
    else:
        electrode_positions[i,:,0] = np.arange(0, 491, 70)
    i+=1

    
#RGCs ----------------------------------------------------------

#on parasol rgc's:

#512 micron patch of retina, 28x32 hexagonal grid (flat hexagon, tops/bottoms flat, sides pointy)
#28.5*height of hexagon = 512, h ~ 17.96 microns, y value increases by 1/2 h each row
#16*(1.5*width of hexagon)+3/4*width of hexagon = 512, w = 512/24.75 ~ 20.7 microns
#set step size = 1.5*w
#odd rows go x=w/2 to x=512-(w/2+3w/4) by 1.5w, even rows go x=0+(w/2+3w/4) to x=512-w/2 by 1.5w

h=512/28.5
w=512/24.75
y_positions = np.arange(h/2, 1+512-h/2, h/2)

i = 0
while(i<np.shape(on_parasol_positions)[0]): #go through all rows, use np.arrange to evenly space neurons
    #set y position for this row
    on_parasol_positions[i,:,1] = np.ones(16)*y_positions[i]
    
    #set x positions for each column in this row
    if(i%2 == 0):
        on_parasol_positions[i,:,0] = np.arange(w/2+3*w/4, 1+512-w/2, 3*w/2)
    else:
        on_parasol_positions[i,:,0] = np.arange(w/2, 1+512-(w/2+3*w/4), 3*w/2)
    i+=1


#off parasol rgc's:

#512 micron patch of retina, 31x35 hexagonal grid (flat hexagon, tops/bottoms flat, sides pointy)
#31.5*height of hexagon = 512, h ~ 16.25 microns, y value increases by 1/2 h each row
#18*(1.5*width of hexagon) = 512, w = 512/27 ~ 18.96 microns
#set step size = 1.5*w
#odd rows go x=w/2 to x=512-(w/2) by 1.5w, even rows go x=0+(w/2+3w/4) to x=512-w/2 by 1.5w
#(even rows have 1 less hexagon though, we can still place it as normal but set it's response to 0
#so it has no effect on later neurons, additionally ignore extra -3/4 w since 1 less diagonal hexagon)

h=512/31.5
w=512/27
y_positions = np.arange(h/2, 1+512-h/2, h/2)

i = 0
while(i<np.shape(off_parasol_positions)[0]): #go through all rows, use np.arrange to evenly space neurons
    #set y position for this row
    off_parasol_positions[i,:,1] = np.ones(18)*y_positions[i]
    
    #set x positions for each column in this row
    if(i%2 == 0):
        off_parasol_positions[i,:,0] = np.arange(w/2+3*w/4, 1+512-w/2+3*w/2, 3*w/2)
        #add extra 3w/2 so we can have an extra dummy spot for missing hexagon at end
    else:
        off_parasol_positions[i,:,0] = np.arange(w/2, 1+512-(w/2), 3*w/2)
    i+=1



#on midget rgc's:

#512 micron patch of retina, 55x63 hexagonal grid (flat hexagon, tops/bottoms flat, sides pointy)
#55.5*height of hexagon = 512, h ~ 9.23 microns, y value increases by 1/2 h each row
#32*(1.5*width of hexagon) = 512, w = 512/48 ~ 10.67 microns
#set step size = 1.5*w
#odd rows go x=w/2 to x=512-(w/2) by 1.5w, even rows go x=0+(w/2+3w/4) to x=512-w/2 by 1.5w
#(even rows have 1 less hexagon though, we can still place it as normal but set it's response to 0
#so it has no effect on later neurons, additionally ignore extra -3/4 w since 1 less diagonal hexagon)

h=512/55.5
w=512/48
y_positions = np.arange(h/2, 1+512-h/2, h/2)

i = 0
while(i<np.shape(on_midget_positions)[0]): #go through all rows, use np.arrange to evenly space neurons
    #set y position for this row
    on_midget_positions[i,:,1] = np.ones(32)*y_positions[i]
    
    #set x positions for each column in this row
    if(i%2 == 0):
        on_midget_positions[i,:,0] = np.arange(w/2+3*w/4, 1+512-w/2+3*w/2, 3*w/2)
    else:
        on_midget_positions[i,:,0] = np.arange(w/2, 1+512-(w/2), 3*w/2)
    i+=1


#off midget rgc's:

#512 micron patch of retina, 61x70 hexagonal grid (flat hexagon, tops/bottoms flat, sides pointy)
#61.5*height of hexagon = 512, h ~ 8.33 microns, y value increases by 1/2 h each row
#35*(1.5*width of hexagon)+3/4*width of hexagon = 512, w = 512/53.25 ~ 9.62 microns
#set step size = 1.5*w
#odd rows go x=w/2 to x=512-(w/2+3w/4) by 1.5w, even rows go x=0+(w/2+3w/4) to x=512-w/2 by 1.5w

h=512/61.5
w=512/53.25
y_positions = np.arange(h/2, 1+512-h/2, h/2)

i = 0
while(i<np.shape(off_midget_positions)[0]): #go through all rows, use np.arrange to evenly space neurons
    #set y position for this row
    off_midget_positions[i,:,1] = np.ones(35)*y_positions[i]
    
    #set x positions for each column in this row
    if(i%2 == 0):
        off_midget_positions[i,:,0] = np.arange(w/2+3*w/4, 1+512-w/2, 3*w/2)
    else:
        off_midget_positions[i,:,0] = np.arange(w/2, 1+512-(w/2+3*w/4), 3*w/2)
    i+=1
    
    
#Bipolars ----------------------------------------------------------

#on parasol bipolar cells:

#512 micron patch of retina, 256x256 hexagonal grid (flat hexagon, tops/bottoms flat, sides pointy)
#256.5*height of hexagon = 512, h ~ 1.996 microns, y value increases by 1/2 h each row
#128*(1.5*width of hexagon)+3/4*width of hexagon = 512, w = 512/192.75 ~ 2.656 microns
#set step size = 1.5*w
#odd rows go x=w/2 to x=512-(w/2+3w/4) by 1.5w, even rows go x=0+(w/2+3w/4) to x=512-w/2 by 1.5w

h=512/256.5
w=512/192.75
y_positions = np.arange(h/2, 1+512-h/2, h/2)

i = 0
while(i<np.shape(on_parasol_bipolar_positions)[0]): #go through all rows, use np.arrange to evenly space neurons
    #set y position for this row
    on_parasol_bipolar_positions[i,:,1] = np.ones(128)*y_positions[i]
    
    #set x positions for each column in this row
    if(i%2 == 0):
        on_parasol_bipolar_positions[i,:,0] = np.arange(w/2+3*w/4, 1+512-w/2, 3*w/2)
    else:
        on_parasol_bipolar_positions[i,:,0] = np.arange(w/2, 1+512-(w/2+3*w/4), 3*w/2)
    i+=1


#off parasol bipolar cells:

#512 micron patch of retina, 256x256 hexagonal grid (flat hexagon, tops/bottoms flat, sides pointy)
#256.5*height of hexagon = 512, h ~ 1.996 microns, y value increases by 1/2 h each row
#128*(1.5*width of hexagon)+3/4*width of hexagon = 512, w = 512/192.75 ~ 2.656 microns
#set step size = 1.5*w
#odd rows go x=w/2 to x=512-(w/2+3w/4) by 1.5w, even rows go x=0+(w/2+3w/4) to x=512-w/2 by 1.5w

h=512/256.5
w=512/192.75
y_positions = np.arange(h/2, 1+512-h/2, h/2)

i = 0
while(i<np.shape(off_parasol_bipolar_positions)[0]): #go through all rows, use np.arrange to evenly space neurons
    #set y position for this row
    off_parasol_bipolar_positions[i,:,1] = np.ones(128)*y_positions[i]
    
    #set x positions for each column in this row
    if(i%2 == 0):
        off_parasol_bipolar_positions[i,:,0] = np.arange(w/2+3*w/4, 1+512-w/2, 3*w/2)
    else:
        off_parasol_bipolar_positions[i,:,0] = np.arange(w/2, 1+512-(w/2+3*w/4), 3*w/2)
    i+=1

    

#on midget bipolar cells:

#512 micron patch of retina, 256x256 hexagonal grid (flat hexagon, tops/bottoms flat, sides pointy)
#256.5*height of hexagon = 512, h ~ 1.996 microns, y value increases by 1/2 h each row
#128*(1.5*width of hexagon)+3/4*width of hexagon = 512, w = 512/192.75 ~ 2.656 microns
#set step size = 1.5*w
#odd rows go x=w/2 to x=512-(w/2+3w/4) by 1.5w, even rows go x=0+(w/2+3w/4) to x=512-w/2 by 1.5w

h=512/256.5
w=512/192.75
y_positions = np.arange(h/2, 1+512-h/2, h/2)

i = 0
while(i<np.shape(on_midget_bipolar_positions)[0]): #go through all rows, use np.arrange to evenly space neurons
    #set y position for this row
    on_midget_bipolar_positions[i,:,1] = np.ones(128)*y_positions[i]
    
    #set x positions for each column in this row
    if(i%2 == 0):
        on_midget_bipolar_positions[i,:,0] = np.arange(w/2+3*w/4, 1+512-w/2, 3*w/2)
    else:
        on_midget_bipolar_positions[i,:,0] = np.arange(w/2, 1+512-(w/2+3*w/4), 3*w/2)
    i+=1


#off midget bipolar cells:

#512 micron patch of retina, 256x256 hexagonal grid (flat hexagon, tops/bottoms flat, sides pointy)
#256.5*height of hexagon = 512, h ~ 1.996 microns, y value increases by 1/2 h each row
#128*(1.5*width of hexagon)+3/4*width of hexagon = 512, w = 512/192.75 ~ 2.656 microns
#set step size = 1.5*w
#odd rows go x=w/2 to x=512-(w/2+3w/4) by 1.5w, even rows go x=0+(w/2+3w/4) to x=512-w/2 by 1.5w

h=512/256.5
w=512/192.75
y_positions = np.arange(h/2, 1+512-h/2, h/2)

i = 0
while(i<np.shape(off_midget_bipolar_positions)[0]): #go through all rows, use np.arrange to evenly space neurons
    #set y position for this row
    off_midget_bipolar_positions[i,:,1] = np.ones(128)*y_positions[i]
    
    #set x positions for each column in this row
    if(i%2 == 0):
        off_midget_bipolar_positions[i,:,0] = np.arange(w/2+3*w/4, 1+512-w/2, 3*w/2)
    else:
        off_midget_bipolar_positions[i,:,0] = np.arange(w/2, 1+512-(w/2+3*w/4), 3*w/2)
    i+=1


In [4]:
# get electrode voltages from image

# random data:

electrode_voltages = np.random.rand(8,8) * 8 #lets assume up to 8 mV for now
electrode_voltages

array([[7.62959324, 3.36018652, 4.43631363, 4.82739792, 7.24978997,
        4.45113184, 4.89528309, 5.72828733],
       [5.21285943, 0.6610578 , 0.44331962, 4.72600604, 3.95321884,
        1.49554364, 2.28053484, 6.69211432],
       [3.12622901, 7.25597519, 4.6610417 , 3.97374082, 6.5474407 ,
        1.39479521, 0.86769407, 4.63623692],
       [4.74435052, 0.85172725, 5.51987641, 5.03966083, 0.04590821,
        6.13671742, 3.34718956, 0.28041502],
       [3.03185467, 2.09604092, 7.70886645, 1.88838978, 7.25052828,
        4.16085576, 0.48369279, 6.03880425],
       [7.2442411 , 4.72591128, 0.86471204, 7.16728557, 6.47997457,
        2.1246945 , 6.07699742, 6.03307122],
       [4.37455775, 6.24846607, 7.93842674, 7.80960362, 1.15189196,
        0.1409157 , 2.13290389, 6.6318234 ],
       [0.491942  , 7.10863566, 1.41906811, 7.18066324, 2.30845507,
        4.31831595, 7.57954445, 3.00110044]])

In [5]:
# get response of bipolar cells from electrodes


#spacial response

#loop through all electrodes, find "bounding box" (within ~35 microns of electrode in each direction),
#go through all bipolars and add response to that electrode if in box otherwise go to next neuron to speed up simulation

i=0
j=0
while(i<np.shape(electrode_positions)[0]):
    while(j<np.shape(electrode_positions)[1]):
        x_elec = electrode_positions[i,j,0]
        y_elec = electrode_positions[i,j,1]
        x1 = x_elec - 35
        x2 = x_elec + 35
        y1 = y_elec - 35
        y2 = y_elec + 35
        
        voltage = electrode_voltages[i,j]
        
        #go through all neurons and simulate responses
        
        #on parasol bipolar simulation:
        a=0
        b=0
        while(a<np.shape(on_parasol_bipolar_positions)[0]):
            while(b<np.shape(on_parasol_bipolar_positions)[1]):
                x=on_parasol_bipolar_positions[a,b,0]
                y=on_parasol_bipolar_positions[a,b,1]
                if(x1<x and x<x2 and y1<y and y<y2):
                    #add to response since it's inside the range of the electrode
                    
                    #bottom left of golden page 4 sais response is based solely on Gaussian from electrode
                    #also included is std dev = 35 microns
                    #G(x,y) = 1/(2*pi*sigma^2) * e^(-(r^2)/(2*sigma^2))
                    
                    r = math.sqrt((x - x_elec)**2 + (y-y_elec)**2)
                    sigma = 35
                    G = 1/(2*math.pi*sigma**2) * math.e**(-(r**2)/(2*sigma**2))
                    #G = 1 * math.e**(-(r**2)/(2*sigma**2)) # G with amplitude 1
                    
                    on_parasol_bipolar_responses[a,b] += G*voltage
                
                #otherwise just continue to next neuron
                
                b+=1
            b=0
            a+=1
        
        
        #off parasol bipolar simulation:
        a=0
        b=0
        while(a<np.shape(off_parasol_bipolar_positions)[0]):
            while(b<np.shape(off_parasol_bipolar_positions)[1]):
                x=off_parasol_bipolar_positions[a,b,0]
                y=off_parasol_bipolar_positions[a,b,1]
                if(x1<x and x<x2 and y1<y and y<y2):
                    #add to response since it's inside the range of the electrode
                    
                    #bottom left of golden page 4 sais response is based solely on Gaussian from electrode
                    #also included is std dev = 35 microns
                    #G(x,y) = 1/(2*pi*sigma^2) * e^(-(r^2)/(2*sigma^2))
                    
                    r = math.sqrt((x - x_elec)**2 + (y-y_elec)**2)
                    sigma = 35
                    G = 1/(2*math.pi*sigma**2) * math.e**(-(r**2)/(2*sigma**2))
                    #G = 1 * math.e**(-(r**2)/(2*sigma**2)) # G with amplitude 1
                    
                    off_parasol_bipolar_responses[a,b] += G*voltage
                
                #otherwise just continue to next neuron
                
                b+=1
            b=0
            a+=1
        
        
        
        #on midget bipolar simulation:
        a=0
        b=0
        while(a<np.shape(on_midget_bipolar_positions)[0]):
            while(b<np.shape(on_midget_bipolar_positions)[1]):
                x=on_midget_bipolar_positions[a,b,0]
                y=on_midget_bipolar_positions[a,b,1]
                if(x1<x and x<x2 and y1<y and y<y2):
                    #add to response since it's inside the range of the electrode
                    
                    #bottom left of golden page 4 sais response is based solely on Gaussian from electrode
                    #also included is std dev = 35 microns
                    #G(x,y) = 1/(2*pi*sigma^2) * e^(-(r^2)/(2*sigma^2))
                    
                    r = math.sqrt((x - x_elec)**2 + (y-y_elec)**2)
                    sigma = 35
                    G = 1/(2*math.pi*sigma**2) * math.e**(-(r**2)/(2*sigma**2))
                    #G = 1 * math.e**(-(r**2)/(2*sigma**2)) # G with amplitude 1
                    
                    on_midget_bipolar_responses[a,b] += G*voltage
                
                #otherwise just continue to next neuron
                
                b+=1
            b=0
            a+=1
        
        
        #off midget bipolar simulation:
        a=0
        b=0
        while(a<np.shape(off_midget_bipolar_positions)[0]):
            while(b<np.shape(off_midget_bipolar_positions)[1]):
                x=off_midget_bipolar_positions[a,b,0]
                y=off_midget_bipolar_positions[a,b,1]
                if(x1<x and x<x2 and y1<y and y<y2):
                    #add to response since it's inside the range of the electrode
                    
                    #bottom left of golden page 4 sais response is based solely on Gaussian from electrode
                    #also included is std dev = 35 microns
                    #G(x,y) = 1/(2*pi*sigma^2) * e^(-(r^2)/(2*sigma^2))
                    
                    r = math.sqrt((x - x_elec)**2 + (y-y_elec)**2)
                    sigma = 35
                    G = 1/(2*math.pi*sigma**2) * math.e**(-(r**2)/(2*sigma**2))
                    #G = 1 * math.e**(-(r**2)/(2*sigma**2)) # G with amplitude 1
                    
                    off_midget_bipolar_responses[a,b] += G*voltage
                
                #otherwise just continue to next neuron
                
                b+=1
            b=0
            a+=1
        
        j+=1
    j=0
    i+=1

print(on_parasol_bipolar_responses)

#temporal response


#Since golden paper didn't give any information on the temporal filter (and we are using slightly different
#temporal modeling with single frame of picture shown then nothing for the rest of the 0.4 s), we will use
#exponential decay to model the temporal behavior with the initial value starting at the spacial response
#and the decay value to be 50 (0.02s half life of neuron response strength, close to curve of golden
#temporal response)

#response = init_response * e^(decay_val*t)

decay_val = -50


#on parasol temporal:

initial_on_parasol_bipolar_responses = np.copy(on_parasol_bipolar_responses)

t=0.001 #time steps of 0.001s (400 time periods)
while t<0.4: #simulate 0.4s like golden paper did
    i=0
    j=0
    while(i<np.shape(on_parasol_bipolar_positions)[0]):
        while(j<np.shape(on_parasol_bipolar_positions)[1]):
            #calculate current time's temporal response value with exponential decay equation, then add it
            #to the overall bipolar response
            
            init_response = initial_on_parasol_bipolar_responses[i,j]
            
            response = init_response * math.e**(decay_val*t)
            
            on_parasol_bipolar_responses[i,j] += response #sum over temporal response for each neuron
            
            j+=1
        i+=1
    
    t+=0.001


#off parasol temporal:

initial_off_parasol_bipolar_responses = np.copy(off_parasol_bipolar_responses)

t=0.001 #time steps of 0.001s (400 time periods)
while t<0.4: #simulate 0.4s like golden paper did
    i=0
    j=0
    while(i<np.shape(off_parasol_bipolar_positions)[0]):
        while(j<np.shape(off_parasol_bipolar_positions)[1]):
            #calculate current time's temporal response value with exponential decay equation, then add it
            #to the overall bipolar response
            
            init_response = initial_off_parasol_bipolar_responses[i,j]
            
            response = init_response * math.e**(decay_val*t)
            
            off_parasol_bipolar_responses[i,j] += response #sum over temporal response for each neuron
            
            j+=1
        i+=1
    
    t+=0.001


#on midget temporal:

initial_on_midget_bipolar_responses = np.copy(on_midget_bipolar_responses)

t=0.001 #time steps of 0.001s (400 time periods)
while t<0.4: #simulate 0.4s like golden paper did
    i=0
    j=0
    while(i<np.shape(on_midget_bipolar_positions)[0]):
        while(j<np.shape(on_midget_bipolar_positions)[1]):
            #calculate current time's temporal response value with exponential decay equation, then add it
            #to the overall bipolar response
            
            init_response = initial_on_midget_bipolar_responses[i,j]
            
            response = init_response * math.e**(decay_val*t)
            
            on_midget_bipolar_responses[i,j] += response #sum over temporal response for each neuron
            
            j+=1
        i+=1
    
    t+=0.001


#off midget temporal:

initial_off_midget_bipolar_responses = np.copy(off_midget_bipolar_responses)

t=0.001 #time steps of 0.001s (400 time periods)
while t<0.4: #simulate 0.4s like golden paper did
    i=0
    j=0
    while(i<np.shape(off_midget_bipolar_positions)[0]):
        while(j<np.shape(off_midget_bipolar_positions)[1]):
            #calculate current time's temporal response value with exponential decay equation, then add it
            #to the overall bipolar response
            
            init_response = initial_off_midget_bipolar_responses[i,j]
            
            response = init_response * math.e**(decay_val*t)
            
            off_midget_bipolar_responses[i,j] += response #sum over temporal response for each neuron
            
            j+=1
        i+=1
    
    t+=0.001

[[7.84221389e-04 8.27963337e-04 8.62889469e-04 ... 6.48339282e-04
  6.66802341e-04 6.76960824e-04]
 [7.68552393e-04 8.16695349e-04 8.56679402e-04 ... 6.43762912e-04
  6.66399885e-04 6.80950454e-04]
 [8.02322225e-04 8.47073793e-04 8.82806065e-04 ... 6.63303784e-04
  6.82192995e-04 6.92585948e-04]
 ...
 [6.36331851e-05 6.29496561e-05 6.14716261e-05 ... 3.74764015e-04
  3.61151930e-04 3.43552915e-04]
 [6.32123462e-05 6.21294386e-05 6.02787987e-05 ... 3.67440742e-04
  3.51807576e-04 3.32502339e-04]
 [6.32193786e-05 6.25402946e-05 6.10718762e-05 ... 3.72326925e-04
  3.58803360e-04 3.41318791e-04]]


In [8]:
#simulate rgc responses to bipolars

#loop through all bipolars, find "bounding box" (within ~30 microns of electrode in each direction),
#go through all bipolars and add response to that electrode if in box otherwise go to next neuron to speed up simulation

br = 0

#on parasol rgc:

i=0
j=0
while(i<np.shape(on_parasol_bipolar_positions)[0]):
    while(j<np.shape(on_parasol_bipolar_positions)[1]):
        x_elec = on_parasol_bipolar_positions[i,j,0]
        y_elec = on_parasol_bipolar_positions[i,j,1]
        x1 = x_elec - 25 #past 25 microns away is .6% & 1.5% of normal curves (center and surround)
        x2 = x_elec + 25
        y1 = y_elec - 25
        y2 = y_elec + 25
        
        response = on_parasol_bipolar_responses[i,j]
        
        #go through all rgc and simulate responses
        
        #on parasol rgc simulation:
        a=0
        b=0
        while(a<np.shape(on_parasol_positions)[0]):
            while(b<np.shape(on_parasol_positions)[1]):
                x=on_parasol_positions[a,b,0]
                y=on_parasol_positions[a,b,1]
                if(y>=y2):
                    br = 1
                    break
                if(x>=x2):
                    break
                if(x1<x and x<x2 and y1<y and y<y2):
                    #add to response since it's inside the range of the bipolar
                    
                    #Lecture 6: G(x,y) = 1/(2*pi*sigma^2) * e^(-(r^2)/(2*sigma^2))
                    #gaussian center std dev values are 10, 8, 4, and 3.5 microns (on par, off par, on mid, off mid)
                    #gaussian surround std dev values from golden are 1.15 * center std dev
                    #center peak has value 1, surround peak has value 0.375 for parasol and 0.5 for midget
                    #(all from the end of page 3 and beginning of page 4)

                    #gaussian with amplitude included: G(x,y) = a * e^(-(r^2)/(2*sigma^2))

                    #diff of gaussians with amplitude: DoG(x,y) = a_cent * e^(-(r^2)/(2*sig_cent^2))
                    # - a_sur * e^(-(r^2)/(2*sig_sur^2))

                    #r(x,y) = sqrt((x-x_elec)^2 + (y-y_elec)^2)

                    r = math.sqrt((x - x_elec)**2 + (y-y_elec)**2)
                    a_cent = 1
                    a_sur = 0.375
                    sig_cent = 10
                    sig_sur = sig_cent * 1.15
                    DoG = a_cent * math.e**(-(r**2)/(2*sig_cent**2)) - a_sur * math.e**(-(r**2)/(2*sig_sur**2))
                    on_parasol_responses[a,b] += DoG * response
                    
                #otherwise just continue to next neuron
                
                b+=1
            
            if(br==1):
                br = 0
                break
            b=0
            a+=1
        
        j+=1
    j=0
    i+=1



#off parasol rgc:

i=0
j=0
while(i<np.shape(off_parasol_bipolar_positions)[0]):
    while(j<np.shape(off_parasol_bipolar_positions)[1]):
        x_elec = off_parasol_bipolar_positions[i,j,0]
        y_elec = off_parasol_bipolar_positions[i,j,1]
        x1 = x_elec - 20 #past 20 microns away is .6% & 1.5% of normal curves (center and surround)
        x2 = x_elec + 20
        y1 = y_elec - 20
        y2 = y_elec + 20
        
        response = off_parasol_bipolar_responses[i,j]
        
        #go through all rgc and simulate responses
        
        #off parasol rgc simulation:
        a=0
        b=0
        while(a<np.shape(off_parasol_positions)[0]):
            while(b<np.shape(off_parasol_positions)[1]):
                if(a%2 == 0 and b == len(off_parasol_responses[0])-1):
                    #row with 1 less rgc, ignore last neuron column
                    break
                x=off_parasol_positions[a,b,0]
                y=off_parasol_positions[a,b,1]
                if(y>=y2):
                    br = 1
                    break
                if(x>=x2):
                    break
                if(x1<x and x<x2 and y1<y and y<y2):
                    #add to response since it's inside the range of the bipolar
                    
                    #Lecture 6: G(x,y) = 1/(2*pi*sigma^2) * e^(-(r^2)/(2*sigma^2))
                    #gaussian center std dev values are 10, 8, 4, and 3.5 microns (on par, off par, on mid, off mid)
                    #gaussian surround std dev values from golden are 1.15 * center std dev
                    #center peak has value 1, surround peak has value 0.375 for parasol and 0.5 for midget
                    #(all from the end of page 3 and beginning of page 4)

                    #gaussian with amplitude included: G(x,y) = a * e^(-(r^2)/(2*sigma^2))

                    #diff of gaussians with amplitude: DoG(x,y) = a_cent * e^(-(r^2)/(2*sig_cent^2))
                    # - a_sur * e^(-(r^2)/(2*sig_sur^2))

                    #r(x,y) = sqrt((x-x_elec)^2 + (y-y_elec)^2)

                    r = math.sqrt((x - x_elec)**2 + (y-y_elec)**2)
                    a_cent = 1
                    a_sur = 0.375
                    sig_cent = 8
                    sig_sur = sig_cent * 1.15
                    DoG = a_cent * math.e**(-(r**2)/(2*sig_cent**2)) - a_sur * math.e**(-(r**2)/(2*sig_sur**2))
                    off_parasol_responses[a,b] += DoG * response
                    
                #otherwise just continue to next neuron
                
                b+=1
            
            if(br==1):
                br = 0
                break
            b=0
            a+=1
        
        j+=1
    j=0
    i+=1





#on midget rgc:

i=0
j=0
while(i<np.shape(on_midget_bipolar_positions)[0]):
    while(j<np.shape(on_midget_bipolar_positions)[1]):
        x_elec = on_midget_bipolar_positions[i,j,0]
        y_elec = on_midget_bipolar_positions[i,j,1]
        x1 = x_elec - 10 #past 10 microns away is .6% & 1.5% of normal curves (center and surround)
        x2 = x_elec + 10
        y1 = y_elec - 10
        y2 = y_elec + 10
        
        response = on_midget_bipolar_responses[i,j]
        
        #go through all rgc and simulate responses
        
        #on midget rgc simulation:
        a=0
        b=0
        while(a<np.shape(on_midget_positions)[0]):
            while(b<np.shape(on_midget_positions)[1]):
                if(a%2 == 0 and b == len(on_midget_responses[0])-1):
                    #row with 1 less rgc, ignore last neuron column
                    break
                x=on_midget_positions[a,b,0]
                y=on_midget_positions[a,b,1]
                if(y>=y2):
                    br = 1
                    break
                if(x>=x2):
                    break
                if(x1<x and x<x2 and y1<y and y<y2):
                    #add to response since it's inside the range of the bipolar
                    
                    #Lecture 6: G(x,y) = 1/(2*pi*sigma^2) * e^(-(r^2)/(2*sigma^2))
                    #gaussian center std dev values are 10, 8, 4, and 3.5 microns (on par, off par, on mid, off mid)
                    #gaussian surround std dev values from golden are 1.15 * center std dev
                    #center peak has value 1, surround peak has value 0.375 for parasol and 0.5 for midget
                    #(all from the end of page 3 and beginning of page 4)

                    #gaussian with amplitude included: G(x,y) = a * e^(-(r^2)/(2*sigma^2))

                    #diff of gaussians with amplitude: DoG(x,y) = a_cent * e^(-(r^2)/(2*sig_cent^2))
                    # - a_sur * e^(-(r^2)/(2*sig_sur^2))

                    #r(x,y) = sqrt((x-x_elec)^2 + (y-y_elec)^2)

                    r = math.sqrt((x - x_elec)**2 + (y-y_elec)**2)
                    a_cent = 1
                    a_sur = 0.5
                    sig_cent = 4
                    sig_sur = sig_cent * 1.15
                    DoG = a_cent * math.e**(-(r**2)/(2*sig_cent**2)) - a_sur * math.e**(-(r**2)/(2*sig_sur**2))
                    on_midget_responses[a,b] += DoG * response
                    
                #otherwise just continue to next neuron
                
                b+=1
            
            if(br==1):
                br = 0
                break
            b=0
            a+=1
        
        j+=1
    j=0
    i+=1



#off midget rgc:

i=0
j=0
while(i<np.shape(off_midget_bipolar_positions)[0]):
    while(j<np.shape(off_midget_bipolar_positions)[1]):
        x_elec = off_midget_bipolar_positions[i,j,0]
        y_elec = off_midget_bipolar_positions[i,j,1]
        x1 = x_elec - 9 #past 9 microns away is .5% & 1.3% of normal curves (center and surround)
        x2 = x_elec + 9
        y1 = y_elec - 9
        y2 = y_elec + 9
        
        response = off_midget_bipolar_responses[i,j]
        
        #go through all rgc and simulate responses
        
        #off midget rgc simulation:
        a=0
        b=0
        while(a<np.shape(off_midget_positions)[0]):
            while(b<np.shape(off_midget_positions)[1]):
                x=off_midget_positions[a,b,0]
                y=off_midget_positions[a,b,1]
                if(y>=y2):
                    br = 1
                    break
                if(x>=x2):
                    break
                if(x1<x and x<x2 and y1<y and y<y2):
                    #add to response since it's inside the range of the bipolar
                    
                    #Lecture 6: G(x,y) = 1/(2*pi*sigma^2) * e^(-(r^2)/(2*sigma^2))
                    #gaussian center std dev values are 10, 8, 4, and 3.5 microns (on par, off par, on mid, off mid)
                    #gaussian surround std dev values from golden are 1.15 * center std dev
                    #center peak has value 1, surround peak has value 0.375 for parasol and 0.5 for midget
                    #(all from the end of page 3 and beginning of page 4)

                    #gaussian with amplitude included: G(x,y) = a * e^(-(r^2)/(2*sigma^2))

                    #diff of gaussians with amplitude: DoG(x,y) = a_cent * e^(-(r^2)/(2*sig_cent^2))
                    # - a_sur * e^(-(r^2)/(2*sig_sur^2))

                    #r(x,y) = sqrt((x-x_elec)^2 + (y-y_elec)^2)

                    r = math.sqrt((x - x_elec)**2 + (y-y_elec)**2)
                    a_cent = 1
                    a_sur = 0.5
                    sig_cent = 3.5
                    sig_sur = sig_cent * 1.15
                    DoG = a_cent * math.e**(-(r**2)/(2*sig_cent**2)) - a_sur * math.e**(-(r**2)/(2*sig_sur**2))
                    off_midget_responses[a,b] += DoG * response
                    
                #otherwise just continue to next neuron
                
                b+=1
            
            if(br==1):
                br = 0
                break
            b=0
            a+=1
        
        j+=1
    j=0
    i+=1


In [9]:
on_parasol_responses

array([[0.10386555, 0.0555003 , 0.04574983, 0.03770803, 0.0574644 ,
        0.05316232, 0.05684487, 0.06309059, 0.07169613, 0.098967  ,
        0.06046071, 0.06040345, 0.04901613, 0.063003  , 0.05934013,
        0.06585843],
       [0.07094453, 0.07145863, 0.03283258, 0.03506195, 0.03600715,
        0.04845952, 0.03766362, 0.05257761, 0.04467712, 0.07496789,
        0.06924147, 0.04277708, 0.0467308 , 0.04075102, 0.05355047,
        0.04314101],
       [0.0712883 , 0.03755734, 0.03138543, 0.0259107 , 0.03942068,
        0.03646876, 0.03899843, 0.04327821, 0.04935767, 0.06789806,
        0.0417489 , 0.04145549, 0.03361411, 0.0432171 , 0.04070562,
        0.04511667],
       [0.05447491, 0.05389859, 0.02484462, 0.02644996, 0.02721332,
        0.03655668, 0.02939581, 0.04051504, 0.03449351, 0.05725889,
        0.05239352, 0.0325548 , 0.03543352, 0.03121037, 0.04074975,
        0.03394615],
       [0.0508289 , 0.02469202, 0.0207195 , 0.01695217, 0.02552134,
        0.02758581, 0.03230776, 

In [10]:
#convert response to firing rate


#go through all rgc responses and convert response to firing rate

#-----------------------------------------------------------------------------------------------
#important: adjust the exponential function later once voltages are adjusted

#we'll use an exponential function to get spike rate from rgc response (golden got this from pillow et al 2008)
#large rgc response is 0.1, so exponential goes from e^0=1 to e^0.1=1.1
#6 Hz seems to be a large neuron firing rate for an RGC (from other studies), so we'll use this as max_rate
#exponential function: firing_rate = max_rate*10*(e^(response)-1)
#subtract 1 from exponential so we have values starting from 0, multiply by 10 to get our max firing rate
#to be about equal to our inputted max firing rate

max_rate = 6 #6Hz seems to be a very large RGC firing rate

#hopefully the learned reconstruction matrix will be able to compensate for any differences we've made in
#our equations (as long as we use the same equations here and in learning)


#on parasol rgcs:

i=0
j=0
while(i<np.shape(on_parasol_positions)[0]):
    while(j<np.shape(on_parasol_positions)[1]):
        response = on_parasol_responses[i,j]
        
        firing_rate = max_rate*10*(math.e**(response)-1)
        
        if(firing_rate < 0):
            firing_rate = 0
        
        on_parasol_responses[i,j] = firing_rate
        
        j+=1
    j=0
    i+=1



#off parasol rgcs:

i=0
j=0
while(i<np.shape(off_parasol_positions)[0]):
    while(j<np.shape(off_parasol_positions)[1]):
        response = off_parasol_responses[i,j]
        
        firing_rate = max_rate*10*(math.e**(response)-1)
        
        if(firing_rate < 0):
            firing_rate = 0
        
        off_parasol_responses[i,j] = firing_rate
        
        j+=1
    j=0
    i+=1




#on midget rgcs:

i=0
j=0
while(i<np.shape(on_midget_positions)[0]):
    while(j<np.shape(on_midget_positions)[1]):
        response = on_midget_responses[i,j]
        
        firing_rate = max_rate*10*(math.e**(response)-1)
        
        if(firing_rate < 0):
            firing_rate = 0
        
        on_midget_responses[i,j] = firing_rate
        
        j+=1
    j=0
    i+=1



#off midget rgcs:

i=0
j=0
while(i<np.shape(off_midget_positions)[0]):
    while(j<np.shape(off_midget_positions)[1]):
        response = off_midget_responses[i,j]
        
        firing_rate = max_rate*10*(math.e**(response)-1)
        
        if(firing_rate < 0):
            firing_rate = 0
        
        off_midget_responses[i,j] = firing_rate
        
        j+=1
    j=0
    i+=1


In [11]:
on_parasol_responses

array([[6.567077  , 3.42416032, 2.80874964, 2.30567992, 3.54885411,
        3.27604904, 3.50949564, 3.9073993 , 4.45973037, 6.24179182,
        3.73955152, 3.73590179, 3.0142372 , 3.90180222, 3.66816643,
        4.08452971],
       [4.4113003 , 4.44442307, 2.00265076, 2.14103204, 2.19979539,
        2.97917261, 2.3029128 , 3.23906178, 2.74141065, 4.67097265,
        4.30169731, 2.62231256, 2.87039376, 2.4955644 , 3.30061431,
        2.64510683],
       [4.43344685, 2.29629194, 1.91298889, 1.57495781, 2.41247917,
        2.22851396, 2.38613131, 2.653702  , 3.03576304, 4.21537189,
        2.55795843, 2.5396057 , 2.05112681, 2.64987344, 2.49272722,
        2.76899425],
       [3.35915885, 3.32265434, 1.50934946, 1.60817191, 1.65521914,
        2.23398584, 1.7899277 , 2.48081845, 2.10571871, 3.53579512,
        3.22742118, 1.98543045, 2.16412617, 1.9021512 , 2.49548504,
        2.07173348],
       [3.1285714 , 1.49996383, 1.25613863, 1.02580023, 1.55098797,
        1.67818933, 1.97011958, 

In [8]:
# get images from imagenet
